# Training model

In [ ]:
import tensorflow as tf
import os
from keras.utils import img_to_array, load_img
import numpy as np
import cv2 as cv
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

trainPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training'
testPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test'

batchSize = 64      # Reduce value if you have less GPU

# # Read in example image and get shape
# img = load_img(trainPath + "/Quince/0_100.jpg")
# plt.imshow(img)
# plt.show()

# imgA = img_to_array(img)
# print(imgA.shape)        
# Build model
model = Sequential()
model.add(Conv2D(filters=128, kernel_size=3, activation="relu", input_shape=(100,100,3)))
model.add(MaxPooling2D())
model.add(Conv2D(filters=64, kernel_size=3, activation="relu"))
model.add(Conv2D(filters=32, kernel_size=3, activation="relu"))
model.add(MaxPooling2D())
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(5000, activation="relu"))
model.add(Dense(1000, activation="relu"))
model.add(Dense(131, activation="softmax"))

print(model.summary())


# Compile model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Load data
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.3, horizontal_flip=True, vertical_flip=True, zoom_range=0.3)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(trainPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical", shuffle=True)


test_generator = test_datagen.flow_from_directory(testPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical")

stepsPerEpoch = np.ceil(train_generator.samples / batchSize)
validationSteps = np.ceil(test_generator.samples / batchSize)

# Early stopping
stop_early = EarlyStopping(monitor="val_accuracy", patience=5)      # Stop fitting model if it doesn't improve by 5

history = model.fit(train_generator, steps_per_epoch=stepsPerEpoch, epochs=5, validation_data=test_generator, validation_steps=validationSteps, callbacks=[stop_early])

model.save("/kaggle/working/fruits360.h5")     # Add file path to save the model to

# Testing model

In [ ]:
!pip uninstall opencv-python-headless -y 
!pip install opencv-python --upgrade
import tensorflow as tf
import os
from keras.utils import img_to_array, load_img
import numpy as np
import cv2 as cv

# Load model
model = tf.keras.models.load_model("/kaggle/working/fruits360.h5")       # Put in model path here
print(model.summary())

# Load categories
source_folder = "/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test"
categories = os.listdir(source_folder)
# categories = categories.sort()
print(categories)
print(len(categories))      # Should be 131

# Load and prepare image
def prepareImage(path):
    img = load_img(path, target_size=(100,100))
    imgArray = img_to_array(img)
    # print(imgArray.shape)
    imgArray = np.expand_dims(imgArray, axis=0)
    imgArray = imgArray / 255.
    return imgArray

testImgPath = "/kaggle/input/eggplant/eggplant.jpg"
imageForModel = prepareImage(testImgPath)

resultArray = model.predict(imageForModel, verbose=1)
answers = np.argmax(resultArray, axis=1)
print(categories[answers[0]])

# Show image with text below
img = cv.imread(testImgPath)
cv.putText(img, categories[answers[0]],(0,50), cv.FONT_HERSHEY_COMPLEX, 1, (209, 19, 77), 2)
cv.imshow('img', img)
cv.waitKey(0)




# Hyperparameter Tuning

In [ ]:
!pip install tensorflow-gpu==2.10
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target,
                                                    train_size=0.75, test_size=0.25)

tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_digits_pipeline.py')

# Hyperparameter Tuning 2

In [ ]:
import keras_tuner as kt
import tensorflow as tf
import os
from keras.utils import img_to_array, load_img
import numpy as np
import cv2 as cv
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

#class MyHyperModel(kt.HyperModel):
def build(hp):
    model = Sequential()
    model.add(Conv2D(filters=128, kernel_size=3, activation=hp.Choice('activation', ['relu', 'PReLU', 'elu']), input_shape=(100,100,3)))
    model.add(MaxPooling2D())
    model.add(Conv2D(filters=64, kernel_size=3, activation="relu"))
    model.add(Conv2D(filters=32, kernel_size=3, activation="relu"))
    model.add(MaxPooling2D())
    model.add(Dropout(rate=hp.Float('rate',min_value=0.3,max_value=0.8,step=0.1)))
    model.add(Flatten())
    model.add(Dense(units=hp.Int('units',3000,8000,500), activation=hp.Choice('activation_2', ['relu', 'PReLU', 'elu'])))
    model.add(Dense(units=hp.Int('units_2',500,1500,250), activation="relu"))
    model.add(Dense(131, activation="softmax"))
    #model.add(keras.layers.Dense(
     #   hp.Choice('units', [8, 16, 32]),
      #  activation='relu'))
   # model.add(keras.layers.Dense(1, activation='relu'))
    model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=["accuracy"])
    
    print(model.summary())
    return model

#def fit(hp, model):
    # Early stopping
#stop_early = EarlyStopping(monitor="val_accuracy", patience=5)      # Stop fitting model if it doesn't improve by 5

#history = model.fit(train_generator, steps_per_epoch=stepsPerEpoch, epochs=5, validation_data=test_generator, validation_steps=validationSteps, callbacks=[stop_early])


trainPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training'
testPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test'

batchSize = 64      # Reduce value if you have less GPU

# Load data
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.3, horizontal_flip=True, vertical_flip=True, zoom_range=0.3)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(trainPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical", shuffle=True)


test_generator = test_datagen.flow_from_directory(testPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical")
#x = MyHyperMode(kt.HyperModel)
#build(kt.HyperParameters())
#tuner = kt.Hyperband(hypermodel = build, objective="val_accuracy", max_epochs=10, hyperband_iterations=10,seed=1,max_retries_per_trial=0, max_consecutive_failed_trials=3)
tuner = kt.RandomSearch(hypermodel = build, objective="val_accuracy", max_trials=10,executions_per_trial=1,overwrite=False)
#tuner.search_space_summary()
tuner.search(train_generator,epochs=2,validation_data=test_generator)
tuner.results_summary()

# VGG 16

In [ ]:
import tensorflow as tf
import os
from keras.models import Model
from keras.utils import img_to_array, load_img
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
#from livelossplot.inputs.keras import PlotLossesCallback
import numpy as np

#import cv2 as cv
#from keras.models import Sequential
#from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
#import matplotlib.pyplot as plt


BATCH_SIZE = 64

'''train_datagen = ImageDataGenerator(rotation_range=90, 
                                     brightness_range=[0.1, 0.7],
                                     width_shift_range=0.5, 
                                     height_shift_range=0.5,
                                     horizontal_flip=True, 
                                     vertical_flip=True,
                                     validation_split=0.15,
                                     preprocessing_function=preprocess_input) # VGG16 preprocessing

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) # VGG16 preprocessing
'''

# Load data
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.3, horizontal_flip=True, vertical_flip=True, zoom_range=0.3)

test_datagen = ImageDataGenerator(rescale=1./255)

trainPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training'
testPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test'

batchSize = 64      # Reduce value if you have less GPU

train_generator = train_datagen.flow_from_directory(trainPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical", shuffle=True)


test_generator = test_datagen.flow_from_directory(testPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical")

def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):
    """
    Compiles a model integrated with VGG16 pretrained layers
    
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """
    
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = VGG16(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False
            
    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(5000, activation='relu')(top_model)
    top_model = Dense(1000, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    print(model.summary())
    return model

input_shape = (100, 100, 3)
optim_1 = Adam(learning_rate=0.001)
n_classes=131

n_steps = train_generator.samples // batchSize
n_val_steps = test_generator.samples // batchSize
n_epochs = 50

# First we'll train the model without Fine-tuning
vgg_model = create_model(input_shape, n_classes, optim_1, fine_tune=0)


# Early stopping
stop_early = EarlyStopping(monitor="val_accuracy", patience=5)      # Stop fitting model if it doesn't improve by 5

history = vgg_model.fit(train_generator, steps_per_epoch=n_steps, epochs=5, validation_data=test_generator, validation_steps=n_val_steps, callbacks=[stop_early])


# Inception

In [ ]:
import tensorflow as tf
import os
from keras.models import Model
from keras.utils import img_to_array, load_img
from keras.optimizers import Adam
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
#from livelossplot.inputs.keras import PlotLossesCallback
import numpy as np

#import cv2 as cv
#from keras.models import Sequential
#from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
#import matplotlib.pyplot as plt


BATCH_SIZE = 64

'''train_datagen = ImageDataGenerator(rotation_range=90, 
                                     brightness_range=[0.1, 0.7],
                                     width_shift_range=0.5, 
                                     height_shift_range=0.5,
                                     horizontal_flip=True, 
                                     vertical_flip=True,
                                     validation_split=0.15,
                                     preprocessing_function=preprocess_input) # VGG16 preprocessing

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) # VGG16 preprocessing
'''

# Load data
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.3, horizontal_flip=True, vertical_flip=True, zoom_range=0.3)

test_datagen = ImageDataGenerator(rescale=1./255)

trainPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training'
testPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test'

batchSize = 64      # Reduce value if you have less GPU

train_generator = train_datagen.flow_from_directory(trainPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical", shuffle=True)


test_generator = test_datagen.flow_from_directory(testPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical")

def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):
    """
    Compiles a model integrated with VGG16 pretrained layers
    
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """
    
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = InceptionV3(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False
            
    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(5000, activation='relu')(top_model)
    top_model = Dense(1000, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    print(model.summary())
    return model

input_shape = (100, 100, 3)
optim_1 = Adam(learning_rate=0.001)
n_classes=131

n_steps = train_generator.samples // batchSize
n_val_steps = test_generator.samples // batchSize
n_epochs = 50

# First we'll train the model without Fine-tuning
inception_model = create_model(input_shape, n_classes, optim_1, fine_tune=0)


# Early stopping
stop_early = EarlyStopping(monitor="val_accuracy", patience=5)      # Stop fitting model if it doesn't improve by 5

history = inception_model.fit(train_generator, steps_per_epoch=n_steps, epochs=5, validation_data=test_generator, validation_steps=n_val_steps, callbacks=[stop_early])


In [ ]:
#From MachineLearningMastery

# example of creating a CNN with an efficient inception module
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers.merge import concatenate
from keras.utils import plot_model
 
# function for creating a projected inception module
def inception_module(layer_in, f1, f2_in, f2_out, f3_in, f3_out, f4_out):
 # 1x1 conv
 conv1 = Conv2D(f1, (1,1), padding='same', activation='relu')(layer_in)
 # 3x3 conv
 conv3 = Conv2D(f2_in, (1,1), padding='same', activation='relu')(layer_in)
 conv3 = Conv2D(f2_out, (3,3), padding='same', activation='relu')(conv3)
 # 5x5 conv
 conv5 = Conv2D(f3_in, (1,1), padding='same', activation='relu')(layer_in)
 conv5 = Conv2D(f3_out, (5,5), padding='same', activation='relu')(conv5)
 # 3x3 max pooling
 pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(layer_in)
 pool = Conv2D(f4_out, (1,1), padding='same', activation='relu')(pool)
 # concatenate filters, assumes filters/channels last
 layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
 return layer_out
 
# define model input
visible = Input(shape=(256, 256, 3))
# add inception block 1
layer = inception_module(visible, 64, 96, 128, 16, 32, 32)
# add inception block 1
layer = inception_module(layer, 128, 128, 192, 32, 96, 64)
# create model
model = Model(inputs=visible, outputs=layer)
# summarize model
model.summary()
# plot model architecture
plot_model(model, show_shapes=True, to_file='inception_module.png')

# DenseNet

In [ ]:
# DenseNet 121

import tensorflow as tf
import os
from keras.models import Model
from keras.utils import img_to_array, load_img
from keras.optimizers import Adam
from keras.applications.densenet import DenseNet121, preprocess_input # Also have DenseNet169, DenseNet201
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
#from livelossplot.inputs.keras import PlotLossesCallback
import numpy as np

#import cv2 as cv
#from keras.models import Sequential
#from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
#import matplotlib.pyplot as plt


BATCH_SIZE = 64

'''train_datagen = ImageDataGenerator(rotation_range=90, 
                                     brightness_range=[0.1, 0.7],
                                     width_shift_range=0.5, 
                                     height_shift_range=0.5,
                                     horizontal_flip=True, 
                                     vertical_flip=True,
                                     validation_split=0.15,
                                     preprocessing_function=preprocess_input) # VGG16 preprocessing

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) # VGG16 preprocessing
'''

# Load data
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.3, horizontal_flip=True, vertical_flip=True, zoom_range=0.3)

test_datagen = ImageDataGenerator(rescale=1./255)

trainPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training'
testPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test'

batchSize = 64      # Reduce value if you have less GPU

train_generator = train_datagen.flow_from_directory(trainPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical", shuffle=True)


test_generator = test_datagen.flow_from_directory(testPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical")

def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):
    """
    Compiles a model integrated with VGG16 pretrained layers
    
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """
    
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = DenseNet121(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False
            
    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(5000, activation='relu')(top_model)
    top_model = Dense(1000, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    print(model.summary())
    return model

input_shape = (100, 100, 3)
optim_1 = Adam(learning_rate=0.001)
n_classes=131

n_steps = train_generator.samples // batchSize
n_val_steps = test_generator.samples // batchSize
n_epochs = 50

# First we'll train the model without Fine-tuning
dense_net_model = create_model(input_shape, n_classes, optim_1, fine_tune=0)


# Early stopping
stop_early = EarlyStopping(monitor="val_accuracy", patience=5)      # Stop fitting model if it doesn't improve by 5

history = dense_net_model.fit(train_generator, steps_per_epoch=n_steps, epochs=5, validation_data=test_generator, validation_steps=n_val_steps, callbacks=[stop_early])


# Xception

In [ ]:
import tensorflow as tf
import os
from keras.models import Model
from keras.utils import img_to_array, load_img
from keras.optimizers import Adam
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
#from livelossplot.inputs.keras import PlotLossesCallback
import numpy as np

#import cv2 as cv
#from keras.models import Sequential
#from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
#import matplotlib.pyplot as plt


BATCH_SIZE = 64

'''train_datagen = ImageDataGenerator(rotation_range=90, 
                                     brightness_range=[0.1, 0.7],
                                     width_shift_range=0.5, 
                                     height_shift_range=0.5,
                                     horizontal_flip=True, 
                                     vertical_flip=True,
                                     validation_split=0.15,
                                     preprocessing_function=preprocess_input) # VGG16 preprocessing

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) # VGG16 preprocessing
'''

# Load data
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.3, horizontal_flip=True, vertical_flip=True, zoom_range=0.3)

test_datagen = ImageDataGenerator(rescale=1./255)

trainPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training'
testPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test'

batchSize = 64      # Reduce value if you have less GPU

train_generator = train_datagen.flow_from_directory(trainPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical", shuffle=True)


test_generator = test_datagen.flow_from_directory(testPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical")

def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):
    """
    Compiles a model integrated with VGG16 pretrained layers
    
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """
    
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = Xception(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False
            
    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(5000, activation='relu')(top_model)
    top_model = Dense(1000, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    print(model.summary())
    return model

input_shape = (100, 100, 3)
optim_1 = Adam(learning_rate=0.001)
n_classes=131

n_steps = train_generator.samples // batchSize
n_val_steps = test_generator.samples // batchSize
n_epochs = 50

# First we'll train the model without Fine-tuning
xception_model = create_model(input_shape, n_classes, optim_1, fine_tune=0)


# Early stopping
stop_early = EarlyStopping(monitor="val_accuracy", patience=5)      # Stop fitting model if it doesn't improve by 5

history = xception_model.fit(train_generator, steps_per_epoch=n_steps, epochs=5, validation_data=test_generator, validation_steps=n_val_steps, callbacks=[stop_early])


# YOLO

In [ ]:
#MachineLearningMastery

# based on https://github.com/experiencor/keras-yolo3
import struct
import numpy as np
from keras.layers import Conv2D
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import ZeroPadding2D
from keras.layers import UpSampling2D
from keras.layers.merge import add, concatenate
from keras.models import Model
 
def _conv_block(inp, convs, skip=True):
 x = inp
 count = 0
 for conv in convs:
 if count == (len(convs) - 2) and skip:
 skip_connection = x
 count += 1
 if conv['stride'] > 1: x = ZeroPadding2D(((1,0),(1,0)))(x) # peculiar padding as darknet prefer left and top
 x = Conv2D(conv['filter'],
    conv['kernel'],
    strides=conv['stride'],
    padding='valid' if conv['stride'] > 1 else 'same', # peculiar padding as darknet prefer left and top
    name='conv_' + str(conv['layer_idx']),
    use_bias=False if conv['bnorm'] else True)(x)
 if conv['bnorm']: x = BatchNormalization(epsilon=0.001, name='bnorm_' + str(conv['layer_idx']))(x)
 if conv['leaky']: x = LeakyReLU(alpha=0.1, name='leaky_' + str(conv['layer_idx']))(x)
 return add([skip_connection, x]) if skip else x
 
def make_yolov3_model():
 input_image = Input(shape=(None, None, 3))
 # Layer  0 => 4
 x = _conv_block(input_image, [{'filter': 32, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 0},
   {'filter': 64, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 1},
   {'filter': 32, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 2},
   {'filter': 64, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 3}])
 # Layer  5 => 8
 x = _conv_block(x, [{'filter': 128, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 5},
 {'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 6},
 {'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 7}])
 # Layer  9 => 11
 x = _conv_block(x, [{'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 9},
 {'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 10}])
 # Layer 12 => 15
 x = _conv_block(x, [{'filter': 256, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 12},
 {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 13},
 {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 14}])
 # Layer 16 => 36
 for i in range(7):
 x = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 16+i*3},
 {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 17+i*3}])
 skip_36 = x
 # Layer 37 => 40
 x = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 37},
 {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 38},
 {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 39}])
 # Layer 41 => 61
 for i in range(7):
 x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 41+i*3},
 {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 42+i*3}])
 skip_61 = x
 # Layer 62 => 65
 x = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 62},
 {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 63},
 {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 64}])
 # Layer 66 => 74
 for i in range(3):
 x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 66+i*3},
 {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 67+i*3}])
 # Layer 75 => 79
 x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 75},
 {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 76},
 {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 77},
 {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 78},
 {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 79}], skip=False)
 # Layer 80 => 82
 yolo_82 = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 80},
   {'filter':  255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 81}], skip=False)
 # Layer 83 => 86
 x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 84}], skip=False)
 x = UpSampling2D(2)(x)
 x = concatenate([x, skip_61])
 # Layer 87 => 91
 x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 87},
 {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 88},
 {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 89},
 {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 90},
 {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 91}], skip=False)
 # Layer 92 => 94
 yolo_94 = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 92},
   {'filter': 255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 93}], skip=False)
 # Layer 95 => 98
 x = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True,   'layer_idx': 96}], skip=False)
 x = UpSampling2D(2)(x)
 x = concatenate([x, skip_36])
 # Layer 99 => 106
 yolo_106 = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 99},
    {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 100},
    {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 101},
    {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 102},
    {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 103},
    {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 104},
    {'filter': 255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 105}], skip=False)
 model = Model(input_image, [yolo_82, yolo_94, yolo_106])
 return model
 
class WeightReader:
 def __init__(self, weight_file):
 with open(weight_file, 'rb') as w_f:
 major, = struct.unpack('i', w_f.read(4))
 minor, = struct.unpack('i', w_f.read(4))
 revision, = struct.unpack('i', w_f.read(4))
 if (major*10 + minor) >= 2 and major < 1000 and minor < 1000:
 w_f.read(8)
 else:
 w_f.read(4)
 transpose = (major > 1000) or (minor > 1000)
 binary = w_f.read()
 self.offset = 0
 self.all_weights = np.frombuffer(binary, dtype='float32')
 
 def read_bytes(self, size):
 self.offset = self.offset + size
 return self.all_weights[self.offset-size:self.offset]
 
 def load_weights(self, model):
 for i in range(106):
 try:
 conv_layer = model.get_layer('conv_' + str(i))
 print("loading weights of convolution #" + str(i))
 if i not in [81, 93, 105]:
 norm_layer = model.get_layer('bnorm_' + str(i))
 size = np.prod(norm_layer.get_weights()[0].shape)
 beta  = self.read_bytes(size) # bias
 gamma = self.read_bytes(size) # scale
 mean  = self.read_bytes(size) # mean
 var   = self.read_bytes(size) # variance
 weights = norm_layer.set_weights([gamma, beta, mean, var])
 if len(conv_layer.get_weights()) > 1:
 bias   = self.read_bytes(np.prod(conv_layer.get_weights()[1].shape))
 kernel = self.read_bytes(np.prod(conv_layer.get_weights()[0].shape))
 kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
 kernel = kernel.transpose([2,3,1,0])
 conv_layer.set_weights([kernel, bias])
 else:
 kernel = self.read_bytes(np.prod(conv_layer.get_weights()[0].shape))
 kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
 kernel = kernel.transpose([2,3,1,0])
 conv_layer.set_weights([kernel])
 except ValueError:
 print("no convolution #" + str(i))
 
 def reset(self):
 self.offset = 0
 
# define the model
model = make_yolov3_model()
# load the model weights
weight_reader = WeightReader('yolov3.weights')
# set the model weights into the model
weight_reader.load_weights(model)
# save the model to file
model.save('model.h5')

In [ ]:
# based on https://github.com/experiencor/keras-yolo3
from numpy import expand_dims
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
 
# load and prepare an image
def load_image_pixels(filename, shape):
    # load the image to get its shape
    image = load_img(filename)
    width, height = image.size
    # load the image with the required size
    image = load_img(filename, target_size=shape)
    # convert to numpy array
    image = img_to_array(image)
    # scale pixel values to [0, 1]
    image = image.astype('float32')
    image /= 255.0
    # add a dimension so that we have one sample
    image = expand_dims(image, 0)
    return image, width, height
 
# load yolov3 model
model = load_model('model.h5')
# define the expected input shape for the model
input_w, input_h = 416, 416
# define our new photo
photo_filename = 'zebra.jpg'
# load and prepare image
image, image_w, image_h = load_image_pixels(photo_filename, (input_w, input_h))
# make prediction
yhat = model.predict(image)
# summarize the shape of the list of arrays
print([a.shape for a in yhat])